In [2]:
import numpy as np
import pandas as pd



ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [7]:
df_days = pd.read_csv('data/day_point_dataset_testset.csv', sep=';')


In [21]:
df_days['employee'] = df_days['employeeId'].apply(lambda x: '_'.join(x.split('_')[:-1]))
df_days['day'] = df_days['employeeId'].apply(lambda x: x.split('_')[-1])

df_days.head(2)



,Unnamed: 0,employeeId,app,app_durations,app_start_times,app_end_times,start_time,end_time,hours_until_next_workday,workday_duration,employee,day
0,0,w6acyepxg4dprew_1,"['Windows_Explorer', 'Microsoft_Outlook', 'Win...","[0.08053333333333333, 0.90395, 0.20135, 0.2683...","[Timestamp('2024-08-01 12:43:57.302000'), Time...","[Timestamp('2024-08-01 12:44:02.134000'), Time...",2024-08-01 12:43:57.302,2024-08-01 21:12:23.917,15.447825,508.443583,w6acyepxg4dprew,1
1,1,w6acyepxg4dprew_3,"['Search_application', 'dell.com', 'Private_Li...","[0.03356666666666666, 0.0334, 0.11753333333333...","[Timestamp('2024-08-02 12:39:16.087000'), Time...","[Timestamp('2024-08-02 12:39:18.101000'), Time...",2024-08-02 12:39:16.087,2024-08-02 20:54:19.876,63.986687,495.063150,w6acyepxg4dprew,3


In [22]:
df_days['app'] = df_days['app'].apply(eval)
df_days['app_durations'] = df_days['app_durations'].apply(eval)



In [30]:
df_days['app_num'] =  df_days['app_durations'].apply(lambda arr: np.array(1 + np.ceil(np.log2(np.maximum(1, np.array(arr)))), dtype=int))

df_days[['employeeId', 'app', 'app_num']].head(2)



,employeeId,app,app_num
0,w6acyepxg4dprew_1,"[Windows_Explorer, Microsoft_Outlook, Windows_...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,w6acyepxg4dprew_3,"[Search_application, dell.com, Private_Links, ...","[1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, ..."


In [31]:
df_days['app_sequence'] = df_days.apply(lambda row: [app for app, num in zip(row['app'], row['app_num']) for _ in range(num)], axis=1)

# Let's check the result
df_days[['employeeId', 'app_sequence']].head(2)

,employeeId,app_sequence
0,w6acyepxg4dprew_1,"[Windows_Explorer, Microsoft_Outlook, Windows_..."
1,w6acyepxg4dprew_3,"[Search_application, dell.com, Private_Links, ..."


In [32]:
df_days['app_num'].explode().value_counts()


app_num
1     183667
2      17274
3      14508
4       6970
5       3488
6        853
7        325
8        136
9         28
10         1
Name: count, dtype: int64

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from itertools import chain
import numpy as np

# Prepare data
sentences = df_days['app_sequence'].tolist()

# Vocabulary and word-to-index mapping
flattened = list(chain.from_iterable(sentences))
vocab = set(flattened)
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}

# Create training data (context-target pairs)
def generate_training_data(sentences, window_size):
    data = []
    for sentence in sentences:
        for i, target in enumerate(sentence):
            start = max(i - window_size, 0)
            end = min(i + window_size + 1, len(sentence))
            context = [sentence[j] for j in range(start, end) if j != i]
            for ctx in context:
                data.append((word2idx[target], word2idx[ctx]))
    return data

training_data = generate_training_data(sentences, window_size=2)

# Model definition
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.output = nn.Linear(embedding_dim, vocab_size)

    def forward(self, input):
        embed = self.embeddings(input)
        out = self.output(embed)
        return out

# Training
embedding_dim = 10
model = Word2Vec(len(vocab), embedding_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Convert data to torch tensors
data_loader = [(torch.tensor([pair[0]]), torch.tensor([pair[1]])) for pair in training_data]

for epoch in range(10):
    total_loss = 0
    for input_word, target_word in data_loader:
        optimizer.zero_grad()
        output = model(input_word)
        loss = criterion(output, target_word)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Word embeddings
word_embeddings = model.embeddings.weight.data.numpy()
print(f"Word embedding for 'is': {word_embeddings[word2idx['is']]}")

In [34]:
from gensim.models import Word2Vec

# Train Word2Vec model on our app sequences
# min_count=1 ensures we keep all apps, even rare ones
# vector_size=100 is a common choice for embedding dimension
# window=5 looks at 5 apps before and after the target app
model = Word2Vec(sentences=df_days['app_sequence'].tolist(),
                vector_size=100,
                window=5,
                min_count=1,
                workers=4)

# Let's test the model by finding similar apps to a sample app
sample_app = df_days['app_sequence'].iloc[0][0]  # Get first app from first sequence
similar_apps = model.wv.most_similar(sample_app)
print(f"Apps most similar to {sample_app}:")
print(similar_apps)

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.12 from "c:\Users\Nemanja\Documents\GitHub\focus-time\.venv\Scripts\python.exe"
  * The NumPy version is: "2.1.3"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.
